<a href="https://colab.research.google.com/github/alanprawira/WeatherPrediction/blob/main/Weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder

In [ ]:
#Load the dataset
dw   = pd.read_csv("/content/drive/MyDrive/dataset/weather.csv")
dc   = pd.read_csv("/content/drive/MyDrive/dataset/crop.csv")

dw

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain
...,...,...,...,...,...,...
1456,2015-12-27,8.6,4.4,1.7,2.9,rain
1457,2015-12-28,1.5,5.0,1.7,1.3,rain
1458,2015-12-29,0.0,7.2,0.6,2.6,fog
1459,2015-12-30,0.0,5.6,-1.0,3.4,sun


In [ ]:
dc

,CROP TYPE,SOIL TYPE,REGION,TEMPERATURE,WEATHER CONDITION,WATER REQUIREMENT
0,BANANA,DRY,DESERT,10-20,NORMAL,8.750
1,BANANA,DRY,DESERT,10-20,SUNNY,10.250
2,BANANA,DRY,DESERT,10-20,WINDY,9.650
3,BANANA,DRY,DESERT,10-20,RAINY,0.750
4,BANANA,DRY,DESERT,20-30,NORMAL,9.850
...,...,...,...,...,...,...
2875,ONION,WET,HUMID,30-40,RAINY,0.100
2876,ONION,WET,HUMID,40-50,NORMAL,4.625
2877,ONION,WET,HUMID,40-50,SUNNY,6.125
2878,ONION,WET,HUMID,40-50,WINDY,5.625


In [ ]:
#check dulicate
duplicates_count = dw.duplicated().sum()
print(f"The number of duplicates: {duplicates_count}")

The number of duplicates: 0


In [ ]:
duplicates_count = dc.duplicated().sum()
print(f"The number of duplicates: {duplicates_count}")

The number of duplicates: 0


In [ ]:
#check missing data
null_pct = dw.apply(pd.isnull).sum()/dw.shape[0]
null_pct

,0
date,0.0
precipitation,0.0
temp_max,0.0
temp_min,0.0
wind,0.0
weather,0.0


In [ ]:
null_pct = dc.apply(pd.isnull).sum()/dc.shape[0]
null_pct

,0
CROP TYPE,0.0
SOIL TYPE,0.0
REGION,0.0
TEMPERATURE,0.0
WEATHER CONDITION,0.0
WATER REQUIREMENT,0.0


In [ ]:
dw.dtypes

,0
date,object
precipitation,float64
temp_max,float64
temp_min,float64
wind,float64
weather,object


In [ ]:
dc.dtypes

,0
CROP TYPE,object
SOIL TYPE,object
REGION,object
TEMPERATURE,object
WEATHER CONDITION,object
WATER REQUIREMENT,float64


In [ ]:
# Normalize and map weather column
dw['weather'] = dw['weather'].str.lower()
dw['weather'] = dw['weather'].apply(
    lambda x: 'RAINY' if 'rain' in x else ('SUNNY' if 'sun' in x else None))


In [ ]:
# Display the DataFrame after normalization
dw

,date,precipitation,temp_max,temp_min,wind,weather
1,2012-01-02,10.9,10.6,2.8,4.5,RAINY
2,2012-01-03,0.8,11.7,7.2,2.3,RAINY
3,2012-01-04,20.3,12.2,5.6,4.7,RAINY
4,2012-01-05,1.3,8.9,2.8,6.1,RAINY
5,2012-01-06,2.5,4.4,2.2,2.2,RAINY
...,...,...,...,...,...,...
1455,2015-12-26,0.0,4.4,0.0,2.5,SUNNY
1456,2015-12-27,8.6,4.4,1.7,2.9,RAINY
1457,2015-12-28,1.5,5.0,1.7,1.3,RAINY
1459,2015-12-30,0.0,5.6,-1.0,3.4,SUNNY


In [ ]:
# Keep only SUNNY and RAINY
dw = dw[dw['weather'].notnull()]

In [ ]:
# Train weather prediction

x = dw[['precipitation', 'temp_max', 'temp_min', 'wind']]
y = dw['weather']

In [ ]:
# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
# Model training (Random Forest Classifier)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# Model evaluation
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9649805447470817


In [ ]:
# Example prediction
# Replace the values with the features of the sensor

sensor = pd.DataFrame({
    'precipitation': [0],
    'temp_max': [36],
    'temp_min': [30],
    'wind': [2]})



In [ ]:
w_pred = model.predict(sensor)
print("Predicted weather:", w_pred[0])

Predicted weather: SUNNY
